In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

In [ ]:
from navground import sim, core

yaml = """
type: Cross
agent_margin: 0.2
side: 11
target_margin: 0.6
tolerance: 0.4
groups:  
  -
    type: thymio
    number: 22
    control_period: 0.1
    behavior:
      type: HL
      safety_margin: 0.2 
      horizon: 3
      barrier_angle: 1
    radius: 0.2
    kinematics:
      type: 2WDiff
      max_speed: 1.0
      wheel_axis: 2
    state_estimation:
      type: Bounded
      range: 2.0 
"""
scenario = sim.load_scenario(yaml)

In [ ]:
scenario = sim.load_scenario(yaml)
world = sim.World()
scenario.init_world(world, seed=10)

In [ ]:
world.agents[10].behavior.horizon

In [ ]:
world.agents[0].behavior.barrier_angle

In [ ]:
def get_positions(world):
    ps = [] 
    for agent in world.agents:
        ps.append(agent.position)
    return(np.array(ps))

In [ ]:
# Initialize world
world = sim.World()
scenario.init_world(world, seed=10)
# Compute positions
ps = []
ps.append(get_positions(world))
world.run(steps=20, time_step=0.1)
ps.append(get_positions(world))
# Plot figure
fig = plt.figure(figsize=(5,5))
ax = plt.gca()
ax.scatter(ps[0][:,0], ps[0][:,1], s=20, marker="s", c="blue", zorder=2, label="X")
ax.scatter(ps[1][:,0], ps[1][:,1], s=23, marker="x", c="red", zorder=2, label="Y")
for edge in zip(ps[0], ps[1]):
    edge_pts = np.array(edge)
    ax.plot(edge_pts[:,0], edge_pts[:,1], c="gray", zorder=1)

plt.legend()
plt.tight_layout()
plt.savefig("matching-X-Y-positions.png")

Next, we compute both persistent homologies at both timesteps, together with their barcodes and the matrices.

In [ ]:
import iblofunmatch.inter as ibfm
import os
output_dir = "output"
os.makedirs("output", exist_ok=True)

In [ ]:
X = ps[0]
Y = ps[1]
idx_S = list(range(int(X.shape[0])))
# Compute distane matrices
Dist_X = dist.squareform(dist.pdist(X))
Dist_Y = dist.squareform(dist.pdist(Y))
Dist_Z = np.minimum(Dist_X, Dist_Y)
# Compute induced matchings
ibfm_out = [
    ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1),
    ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
]

In [ ]:
def get_matrix(entries_list, num_rows):
    M_X = np.zeros((num_rows, len(entries_list))).astype("int")
    for col_idx, col in enumerate(entries_list):
        for idx in col:
            M_X[idx, col_idx]=1

    return M_X

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(10,3))
ibfm.plot_barcode(ibfm_out[0]["S_barcode_0"], "blue", ax[0])
M = get_matrix(ibfm_out[0]["pm_matrix_0"], ibfm_out[0]["X_barcode_0"].shape[0])
ax[1].matshow(M, cmap="Grays")
ax[1].set_xticks([])
ax[1].set_yticks([])
ibfm.plot_barcode(ibfm_out[0]["X_barcode_0"], "black", ax[2])
M = get_matrix(ibfm_out[1]["pm_matrix_0"], ibfm_out[1]["X_barcode_0"].shape[0])
ax[3].matshow(M, cmap="Grays")
ax[3].set_xticks([])
ax[3].set_yticks([])
ibfm.plot_barcode(ibfm_out[1]["S_barcode_0"], "red", ax[4])
### Print titles 
ax[0].set_title("PH_0(X) barcode")
ax[1].set_title("Matrix F")
ax[2].set_title("PH_0(XuY) barcode")
ax[3].set_title("Matrix G")
ax[4].set_title("PH_0(Y) barcode")
plt.savefig("barcode_0_X_Y_matrices.png")

Next, we depict matching, as well as back to back plot.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,3))
ibfm.plot_matching(ibfm_out[0], ax, fig, max_rad=-1, dim=0, block_function=True)
plt.savefig("matchings_0_X_Z.png")

In [ ]:
a = ibfm_out[0]["S_barcode_0"][-2][1]
b = ibfm_out[0]["X_barcode_0"][
    ibfm_out[0]["block_function_0"][-2]
][1]
a

In [ ]:
fig, ax = plt.subplots(figsize=(5,7))
ibfm.plot_XYZ_matching_0(ibfm_out, ax)
plt.savefig("matchings_XYZ_illustration.png")

# Perform analysis

Measure impact of variables in overall matching performance.

In [ ]:
# Experiment deadlock
num_steps = 400
yaml = f"""
steps: {num_steps}
time_step: 0.1
record_safety_violation: true
record_task_events: true
record_pose: true
runs: 1
scenario:
  type: Cross
  radius: 5
  side: 11
  agent_margin: 0.2
  add_safety_to_agent_margin: true
  tolerance: 0.5
  position_noise: 0.2
  groups:
    - number: 18
      type: thymio
      control_period: 0.1
      behavior:
        type: HL
        safety_margin: 0.8
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
num_steps = 400
yaml = f"""
steps: {num_steps}
time_step: 0.1
record_safety_violation: true
record_task_events: true
record_pose: true
runs: 1
scenario:
  type: Cross
  radius: 5
  side: 11
  agent_margin: 0.2
  add_safety_to_agent_margin: true
  tolerance: 0.5
  position_noise: 0.2
  groups:
    - number: 18
      type: thymio
      control_period: 0.1
      behavior:
        type: HL
        safety_margin: 0.3
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
print(f"Performed {len(experiment.runs)} runs in {experiment.duration.total_seconds()} seconds")

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
start_ps = 120
ps = ps[start_ps:]
ps.shape

Plot poses between timesteps 100 and 120. Together with matching diagrams.

In [ ]:
def plot_two_timesteps(X, Y, ax):
    # Plot figure
    ax.scatter(X[:,0], X[:,1], s=20, marker="s", c="blue", zorder=2, label="X")
    ax.scatter(Y[:,0], Y[:,1], s=23, marker="x", c="red", zorder=2, label="Y")
    for edge in zip(X, Y):
        edge_pts = np.array(edge)
        ax.plot(edge_pts[:,0], edge_pts[:,1], c="gray", zorder=1)
    

In [ ]:
def plot_divergence_diagram(X, Y, ax):
    idx_S = list(range(int(X.shape[0])))
    # Compute distane matrices
    Dist_X = dist.squareform(dist.pdist(X))
    Dist_Y = dist.squareform(dist.pdist(Y))
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    # Compute induced matchings
    ibfm_out = [
        ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1),
        ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    ]
    # Divergence diagrams 
    ibfm.plot_XYZ_matching_0(ibfm_out, ax)
    ax.set_xlim([-4,4])
    # print persistence divergence 
    matching_XZ = ibfm_out[0]["induced_matching_0"]
    matching_YZ = ibfm_out[1]["induced_matching_0"]
    composition_XY = [matching_YZ.index(i) for i in matching_XZ]
    endpoints_0 = np.array(ibfm_out[0]["S_barcode_0"][:,1])
    endpoints_1 = np.array(ibfm_out[1]["S_barcode_0"][:,1])
    endpoints_1 = endpoints_1[composition_XY]
    persistence_divergence = np.sum(np.sqrt((endpoints_0-endpoints_1)**2))
    print(np.abs(endpoints_0-endpoints_1))
    print(persistence_divergence)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
X = ps[200]
Y = ps[220]
plot_two_timesteps(X, Y, ax[0])
plot_divergence_diagram(X, Y, ax[1])
ax[0].legend()
plt.tight_layout()
plt.savefig("persistence_divergence_diagram.png")

Now, we measure the persistence divergence and interpret it in terms of changes between point clouds, efficiency, collisions and deadlocks

In [ ]:
persdiv_list=[] 
shift = 30
idx_S = list(range(int(ps[0].shape[0])))
for start_timestep in range(num_steps-shift-start_ps):
    X = ps[start_timestep]
    Y = ps[start_timestep + shift]
    # Compute distane matrices
    Dist_X = dist.squareform(dist.pdist(X))
    Dist_Y = dist.squareform(dist.pdist(Y))
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    # Compute induced matchings
    ibfm_out = [
        ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, points=False, max_dim=1),
        ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, points=False, max_dim=1)
    ]
    # Compute persistence divergence
    matching_XZ = ibfm_out[0]["induced_matching_0"]
    matching_YZ = ibfm_out[1]["induced_matching_0"]
    composition_XY = [matching_YZ.index(i) for i in matching_XZ]
    endpoints_0 = np.array(ibfm_out[0]["S_barcode_0"][:,1])
    endpoints_1 = np.array(ibfm_out[1]["S_barcode_0"][:,1])
    endpoints_1 = endpoints_1[composition_XY]
    persistence_divergence = np.sum(np.sqrt((endpoints_0-endpoints_1)**2))
    # Store 
    persdiv_list.append(persistence_divergence)

In [ ]:
persdiv_list

In [ ]:
matching_XZ = ibfm_out[0]["induced_matching_0"]
matching_YZ = ibfm_out[1]["induced_matching_0"]
composition_XY = [matching_YZ.index(i) for i in matching_XZ]
print(matching_XZ)
print(matching_YZ)
print(composition_XY)

Now we plot the persistence divergences.

In [ ]:
average_range = 2*shift
divergences_averages = [np.average(persdiv_list[i: i+average_range]) for i in range(len(persdiv_list)-average_range)]
divergences_averages = persdiv_list

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(list(range(len(divergences_averages))), divergences_averages, c="black", label=f"persistence divergence")
plt.legend()

We can check that the point $380$ with higher persistene divergence changes corresponds to a bit chaothic movement of points.

In [ ]:
timestep = np.argmax(divergences_averages)
X = ps[timestep]
Y = ps[timestep + shift]
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
plot_two_timesteps(X, Y, ax[0])
plot_divergence_diagram(X, Y, ax[1])
ax[0].legend()
plt.savefig("max_divergence.png")

In [ ]:
timestep = np.argmin(divergences_averages)
X = ps[timestep]
Y = ps[timestep + shift]
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
plot_two_timesteps(X, Y, ax[0])
plot_divergence_diagram(X, Y, ax[1])
ax[0].legend()
plt.savefig("min_divergence.png")

On the other hand, the point with less persistence divergence, corresponds to the point with a very ordered movement of agents.

In [ ]:
timestep = np.argmin(divergences_averages)
X = ps[timestep]
Y = ps[timestep + shift]
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
plot_two_timesteps(X, Y, ax[0])
plot_divergence_diagram(X, Y, ax[1])
ax[0].legend()
plt.savefig("min_divergence.png")

Let us now do this for the $n$ largest local maxima and the $n$ smallest local minima.

In [ ]:
timestep_list = list(range(50,120,50))
fig, ax = plt.subplots(nrows=len(timestep_list), ncols=2, figsize=(10,5*len(timestep_list)))
for idx, timestep in enumerate(timestep_list):
    X = ps[timestep]
    Y = ps[timestep + shift]
    plot_two_timesteps(X, Y, ax[idx, 0])
    plot_divergence_diagram(X, Y, ax[idx,1])
    ax[idx,0].legend()

plt.savefig("exploration_matchings.png")
# plt.savefig("deadlock_matchings.png")

In [ ]:
n = 4
from scipy.signal import argrelextrema
persdiv_arr = np.array(persdiv_list)
local_maxima = argrelextrema(persdiv_arr, np.greater)[0]
local_minima = argrelextrema(persdiv_arr, np.less)[0]
### Take n largest and n smallest maxima/minima
local_maxima = local_maxima[np.argsort(-persdiv_arr[local_maxima])[:n]]
local_minima = local_minima[np.argsort(persdiv_arr[local_minima])[:n]]

local_maxima.sort()
local_minima.sort()

In [ ]:
# local_maxima.sort()
# local_minima = [i + np.argmin(persdiv_arr[i:j]) for i,j in zip(local_maxima[:-1], local_maxima[1:])]

In [ ]:
local_maxima

In [ ]:
local_minima

We plot these in the persistence divergence function.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(list(range(len(persdiv_list))), persdiv_list, c="black", label=f"persistence divergence")
ax.scatter(local_maxima, persdiv_arr[local_maxima], c="red", label="local maxima")
ax.scatter(local_minima, persdiv_arr[local_minima], c="blue", label="local minima")
plt.legend()

In [ ]:
local_maxima

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=(n-1), figsize=(5*(n-1), 10))
for idx, max_step in enumerate(local_maxima[:-1]):
    X = ps[max_step]
    Y = ps[max_step + shift]
    plot_two_timesteps(X, Y, ax[0, idx])
for idx, min_step in enumerate(local_minima[:-1]):
    X = ps[min_step]
    Y = ps[min_step + shift]
    plot_two_timesteps(X, Y, ax[1, idx])

In [ ]:
persdiv_arr[local_maxima]

In [ ]:
persdiv_arr[local_minima]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=(n-1), figsize=(5*(n-1), 10))
for idx, max_step in enumerate(local_maxima[:-1]):
    X = ps[max_step]
    Y = ps[max_step + shift]
    plot_divergence_diagram(X, Y, ax[0,idx])

print()
for idx, min_step in enumerate(local_minima[:-1]):
    X = ps[min_step]
    Y = ps[min_step + shift]
    plot_divergence_diagram(X, Y, ax[1,idx])